In [114]:
import numpy as np
import mne
import os
from sklearn.model_selection import train_test_split
import h5py
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import torch
from keras.models import load_model
import numpy as np
from sklearn.metrics import mean_squared_error

In [11]:
def metric(y_true, y_pred):
    l2ratio = np.sum((y_true - y_pred)**2) / np.sum(y_true**2)
    return np.sqrt(l2ratio) * 100

In [4]:
def load_file(fnames):
    import h5py
    try:
        h5_file = h5py.File(fnames, 'r')
        a_group_key = list(h5_file.keys())[0]
        eeg_data = np.array(h5_file[a_group_key]).T
    except FileNotFoundError:
        eeg_data = None
    return [eeg_data, eeg_data.shape[0]]

In [6]:
train_dir = "/yakushin/eeg/data/train/"
train_names = [x for x in os.listdir(train_dir) 
                 if x[-3:] == ".h5"]
data = [x for x in load_file.map([train_dir + f for f in train_names]) if x is not None]

In [7]:
X_train = np.concatenate([x[0] for x in data], axis = 0)
X_train_size = np.cumsum(np.array([x[1] for x in data]))
X_train_delete_channels = find_bad_channels(X_train, X_train_size)
del data

In [6]:
def load_new_data():
    eeg_dir = "/yakushin/eeg/data/test/"
    eeg_names = [x for x in os.listdir(eeg_dir) 
                    if x[-3:] == ".h5"]
    data = np.concatenate([x for x in 
                                load_file.map([eeg_dir + f for f in eeg_names]) 
                                    if x is not None], axis=0)
    return np.concatenate([x[0] for x in data], axis = 0), np.cumsum(np.array([x[1] for x in data]))

In [7]:
X_test, X_test_size = load_new_data()
X_test_delete_channels = find_bad_channels(X_test, X_test_size)
X_test_delete_channels = []
start = 0
for finish in X_test_size:
    part_data = X_test[start : finish]
    X_train_delete_channels.append([])
    for i in range(58):
        part_data[:, i]
        if (np.array_equal(part_data[:, i],np.zeros(finish - start))):
            X_train_delete_channels[-1].append(i)
    start = finish

In [8]:
class L2RatioTester(object):
    def metric(self, y_true, y_pred):
        l2ratio = np.sum((y_true - y_pred)**2) / np.sum(y_true**2)
        return np.sqrt(l2ratio) * 100

In [8]:
components_count = components_count = np.logspace(1, 58, 11)

In [11]:
def calc_metric(n_components):
    from sklearn.decomposition import PCA
    pca = PCA(n_components=n_components)
    pca.fit(X_train)
    return (tester.metric(X_test, pca.inverse_transform(pca.transform(X_test)))
    , tester.metric(X_train, pca.inverse_transform(pca.transform(X_train))))

In [12]:
lview['X_train'] = X_train[:200000]
lview['X_test'] = X_test[:200000]
lview['L2RatioTester'] = L2RatioTester
lview['tester'] = L2RatioTester()
l2_ratio = calc_metric.map(components_count)